In [1]:
# import psycopg2
# from y import create_engine
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import fiona
from osgeo import gdal

------------------

In [2]:
general_path = 'E:\paper2_analysis/traffic_analysis/Omnitrans_denHelder'
figures_path = f'{general_path}/figures'
input_path = f'{general_path}/input'
output_path = f'{general_path}/output'

zone_data_path = 'E:/paper2_analysis/depth_arrival_zones/output'

-----------------------------

### zonal data

In [3]:
#from cbs data
buurten_all_centroids = gpd.read_file(f'{input_path}/buurten_centroids.shp')
# buurten_all_vlak = gpd.read_file(f'{input_path}/buurten_vlak.shp')
buurten_all_vlak = gpd.read_file(f'{input_path}/household_data_select_2.geojson')

buurten_all_vlak['id_new'] = np.arange(1, len(buurten_all_vlak)+1)

In [4]:
buurten_all_vlak.columns

Index(['BU_CODE', 'JRSTATCODE', 'BU_NAAM', 'WK_CODE', 'WK_NAAM', 'GM_CODE',
       'GM_NAAM', 'POSTCODE', 'BEV_DICHTH', 'AANT_INW', 'AANTAL_HH',
       'P_EENP_HH', 'P_HH_Z_K', 'P_HH_M_K', 'GEM_HH_GR', 'P_WEST_AL',
       'P_N_W_AL', 'ID', 'WijkenEnBuurten', 'Gemeentenaam_1', 'Codering_3',
       'AantalInwoners_5', 'HuishoudensTotaal_28',
       'GemiddeldeHuishoudensgrootte_32', 'PersonenautoSTotaal_100',
       'PersonenautoSPerHuishouden_103', 'geometry', 'id_new'],
      dtype='object')

In [5]:
buurten_OD_info =  buurten_all_vlak[['id_new', 'BU_NAAM', 'AANTAL_HH','AANT_INW',
                                    'HuishoudensTotaal_28', 'PersonenautoSTotaal_100']]
print(len(buurten_OD_info))
buurten_OD_info.head(2)

79


,id_new,BU_NAAM,AANTAL_HH,AANT_INW,HuishoudensTotaal_28,PersonenautoSTotaal_100
0,1,Oostsloot,650,1115,650,465
1,2,Centrum,870,1405,870,515


In [6]:
total_hh = buurten_OD_info.AANTAL_HH.sum()
total_pv = buurten_OD_info.PersonenautoSTotaal_100.sum() #private vehicles
print(f'{total_hh} sum hh ')
print(f'{total_pv} sum private vehicles ')

34680 sum hh 
35300 sum private vehicles 


--------

### SELECT ZONES FOR DEPARTURE

In [50]:
## selection based on zonal script on flood threshold
scenario_name = 'scen1_voorpiek'
nr_zones_sampling = 89
samplesize_nr_houses_in_zone = 50

threshold_depth = 1.5
threshold_min_start = 120
threshold_min_end = 9999


if scenario_name == 'scen1_voorpiek':
    flood = 1
elif scenario_name == 'scen2_napiek':
    flood = 2
    
simulationcode = f'f{flood}_d{threshold_depth}_interval'

#alleen op diepte
# threshold_calc = f'/zones_maxdepth_threshold_{scenario_name}_sample{samplesize_nr_houses_in_zone}houses_tresh_{threshold_depth}m'

#                 'zones_thrs_depth_timeinterval_scen1_voorpiek_tresh_0m_30min_60min
threshold_calc = f'zones_thrs_depth_timeinterval_{scenario_name}_tresh_{threshold_depth}m_{threshold_min_start}min_{threshold_min_end}min'

In [51]:
simulationcode

'f1_d1.5_interval'

In [52]:
selection_area =  threshold_calc




In [53]:
names_buurtSelection = gpd.read_file(f'{zone_data_path}/__time_intervals/{threshold_calc}.geojson').BU_NAAM.values

----------------

### selection based on zonal script

In [54]:
## load selection made in 
if selection_area ==  threshold_calc:
    print(' threshold_calc selected')
    
    names_buurtSelection = gpd.read_file(f'{zone_data_path}/__time_intervals/{threshold_calc}.geojson').BU_NAAM.values
    buurtSelect = buurten_all_vlak[buurten_all_vlak["BU_NAAM"].isin(names_buurtSelection)]
    
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANT_INW', 'AANTAL_HH', 'PersonenautoSTotaal_100']]
    
    print(f'{len(buurtSelect)} zones selected')
else:
    print("zuidrand_denhelder not selected")


 threshold_calc selected
20 zones selected


In [55]:
# on ids
list_id_in_selection = buurtSelect.id_new.values
print(f'total nr of selected zones: {len(list_id_in_selection)}')
print(list_id_in_selection)

total nr of selected zones: 20
[41 42 44 47 49 50 51 52 53 54 55 56 57 58 59 60 70 71 73 74]


In [56]:
# on names
print(buurtSelect.BU_NAAM.values)
print('--------------------')
print(f'total nr of selected zones: {len(buurtSelect)}')
print(f'aantal HH: {sum(buurtSelect.AANTAL_HH)}')
print(f'aantal INW: {sum(buurtSelect.AANT_INW)}')
print(f'PersonenautoSTotaal_100: {sum(buurtSelect.PersonenautoSTotaal_100)}')

['Julianadorp-Oost' 'Oostoever' 'Kooypunt' 'Koegras-Zuid' 'De Zandloper'
 'Ooghduyne' 'Julianadorp-West' 'Middelzand' 'Vogelzand' 'Doorzwin'
 'Kruiszwin' 'Wierbalg' 'Malzwin' 'Boterzwin' 'Zwanenbalg' 'Noorderhaven'
 'Callantsoog (woonkern)' 'Uyterland' 'Buitengebied Callantsoog-Oost'
 'Buitengebied Callantsoog-Noord']
--------------------
total nr of selected zones: 20
aantal HH: 7040
aantal INW: 16665
PersonenautoSTotaal_100: 8955


------------------------------

### create list zones for OD based on selection

In [57]:
buurten_OD_info

,id_new,BU_NAAM,AANTAL_HH,AANT_INW,HuishoudensTotaal_28,PersonenautoSTotaal_100
0,1,Oostsloot,650,1115,650,465
1,2,Centrum,870,1405,870,515
2,3,Willemsoord,245,415,245,175
3,4,Sluisdijk,555,1010,555,350
4,5,Visbuurt,1230,2405,1230,965
...,...,...,...,...,...,...
74,75,Kleine Sluis inclusief Spoorbuurt,3235,7510,3235,3830
75,76,Gelderse Buurt en omgeving,315,775,315,410
76,77,Breezand,1410,3385,1410,1930
77,78,Van Ewijcksluis,100,205,100,165


In [58]:
simulationcode


'f1_d1.5_interval'

In [59]:
OD_list = buurten_OD_info.copy()

#als id buurten niet in id select zit, dan waarde aanpassen naar nul
for index, row in OD_list.iterrows():
    if row.id_new not in list_id_in_selection:
        OD_list.loc[index,"AANTAL_HH"] = 0
        OD_list.loc[index,"AANT_INW"] = 0
        OD_list.loc[index,"PersonenautoSTotaal_100"] = 0
        
OD_list.to_csv(f'{input_path}/{simulationcode}/OD_selection_{selection_area}.csv', header= True, sep =';', decimal=',' )
OD_list.to_json(f'{input_path}/{simulationcode}/OD_selection_{selection_area}.json')

print(f'{OD_list.AANTAL_HH.sum()} aantal huishoudens')
print(f'{OD_list.PersonenautoSTotaal_100.sum()} PersonenautoSTotaal_100')

7040 aantal huishoudens
8955 PersonenautoSTotaal_100


In [60]:
## CREATE LIST OF Origns NOT in selection
OD_list_notselected = buurten_OD_info.copy()

#als id buurten WEL id select zit, dan waarde aanpassen naar nul
for index, row in OD_list_notselected.iterrows():
    if row.id_new in list_id_in_selection:
        OD_list_notselected.loc[index,"AANTAL_HH"] = 0


# OD_list_notselected.to_csv(f'{input_path}/{simulationcode}/OD_NOTinselection_{selection_area}.csv', header= True, sep =';', decimal=',' )
# OD_list_notselected.to_json(f'{input_path}/{simulationcode}/OD_NOTselection_{selection_area}.json')
print(f'{OD_list_notselected.AANTAL_HH.sum()} aantal huishoudens - buiten selection')

27640 aantal huishoudens - buiten selection


In [61]:
# check 
total_selected = OD_list.AANTAL_HH.sum()
total_outside_selection = OD_list_notselected.AANTAL_HH.sum()
total_check = total_selected + total_outside_selection
total_check == total_hh

True

In [62]:
print(f'total householdes in area: {total_hh}')
print(f'total hh selected: {total_selected}')
print(f'total hh outside selection: {total_outside_selection}')
print('-----------')

if total_hh == (total_selected + total_outside_selection):
    print('check, total klopt')
else:
    print('error')

total householdes in area: 34680
total hh selected: 7040
total hh outside selection: 27640
-----------
check, total klopt


----------------------

### old - via shapefile

### zuidrand

In [ ]:
## load selection made in 
if selection_area == 'zuidrand_denhelder':
    print('zuidrand_denhelder selected')
    
    buurtSelect= gpd.read_file(f'{input_path}/zuidrand_denhelder.shp')
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANTAL_HH']]
    
    print(f'{len(buurtSelect)} zones selected')
else:
    print("zuidrand_denhelder not selected")

### everyone

In [ ]:
## load selection made in 
if selection_area == 'everyone':
    print('entire area is selected')
    
    buurtSelect= gpd.read_file(f'{input_path}/buurten_centroids.shp')
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANTAL_HH']]
    
    print(f'{len(buurtSelect)} zones are selected')
else:
    print("not entire area")

### free selection

In [ ]:
## load selection made in 
if selection_area == 'free_selection':
    print('free_selection made')
    
    buurtSelect= gpd.read_file(f'{input_path}/free_selection2.shp')
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANTAL_HH']]
    
    print(f'{len(buurtSelect)} zones selected')
else:
    print("free_selection not made")

#### DEN HELDER ONLY


In [ ]:
## load selection made in 
if selection_area == 'Den Helder':
    print('Den Helder selected')
    
    buurtSelect= gpd.read_file(f'{input_path}/DenHelder_only.shp')
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANTAL_HH']]
    
    print(f'{len(buurtSelect)} zones selected')
else:
    print("den helder not selected")

### juliadorp only

In [ ]:
## load selection 
if selection_area == 'Julianadorp':
    print('Julianadorp ')
    buurtSelect = gpd.read_file(f'{input_path}/Julianadorp_only.shp')
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANTAL_HH']]

    print(f'{len(buurtSelect)} zones selected')
else:
    print("julianadorp not selected")

### selection based on flood depth

In [ ]:
## load selection 

if selection_area == 'flooddepth_2.5':
    print('flood depth selection_area')
    buurtSelect = gpd.read_file(f'{input_path}/maxdepth_larger2.5.shp')
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANTAL_HH']]

    print(f'{len(buurtSelect)} zones selected')
    print(f'{buurtSelect.AANTAL_HH.sum()} HH selected')
else:
    print("flooddepth areas not selected")

In [ ]:
## load selection 

if selection_area == 'flooddepth_2m':
    print('flood depth selection_area')
    buurtSelect = gpd.read_file(f'{input_path}/maxdepth_larger2.shp')
    buurtSelect = buurtSelect[['id_new', 'BU_NAAM', 'AANTAL_HH']]

    print(f'{len(buurtSelect)} zones selected')
    print(f'{buurtSelect.AANTAL_HH.sum()} HH selected')
else:
    print("flooddepth areas not selected")

In [ ]:
## selection based on shapefile_input - testphase

# selection_area = 'Den Helder'
# selection_area = 'Julianadorp'
# selection_area = "flooddepth_2m"
# selection_area = "flooddepth_2.5"
# selection_area = "free_selection"
# selection_area = 'everyone'
# selection_area = 'zuidrand_denhelder'
